In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations

In [2]:
db_raw = pd.read_excel('salesCG201718.xls', header=None)

In [3]:
db_data_month = db_raw.drop([0,1], axis=0)
db_data_month['month'] = db_data_month[0].dt.month
sales_month = db_data_month.groupby('month').sum()

In [4]:
shops = db_raw.transpose()[[0,1]].drop([0], axis=0)
shops.rename(columns = {0:'shop', 1:'product'}, inplace=True)

for i in range(1,21):
    shops['shop'][2*i] = shops['shop'][2*i-1]

In [5]:
tr_sales_month = sales_month.transpose()
monthly_sales_per_shop = pd.concat([shops, tr_sales_month], axis=1).set_index(['shop', 'product']).transpose()

In [6]:
av_daily_per_month = monthly_sales_per_shop/(6*4)

In [7]:
rounded_av_daily_per_month = av_daily_per_month.apply(np.round)

In [8]:
for i in range(1,13,1):
    monthly_per_month = av_daily_per_month.transpose()[i]

In [9]:
for num_s in range(1,21,1):
    rounded_av_daily_per_month[('S'+str(num_s), 'P1')] = rounded_av_daily_per_month[('S'+str(num_s), 'P1')]*0.8
    rounded_av_daily_per_month[('S'+str(num_s), 'P2')] = rounded_av_daily_per_month[('S'+str(num_s), 'P2')]*0.4

In [10]:
vol_av_daily_per_month = rounded_av_daily_per_month.transpose().groupby('shop').sum().reindex(monthly_per_month.index.get_level_values(0)).drop_duplicates().copy()

In [11]:
vol_av_daily_per_month

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
shop,,,,,,,,,,,,
S1,4.4,4.0,4.8,6.8,6.4,6.4,6.4,6.8,5.6,5.6,3.6,2.8
S2,4.0,4.0,4.8,6.0,7.2,4.8,6.4,6.8,4.8,5.6,4.0,4.0
S3,5.2,6.0,6.8,8.8,8.0,8.0,8.8,8.0,7.6,7.2,5.6,4.8
S4,6.0,5.2,7.6,7.2,8.8,8.8,7.2,8.0,6.8,6.4,4.8,4.4
S5,4.0,4.8,6.8,5.2,7.2,5.6,6.4,7.6,4.8,5.2,4.0,3.2
S6,6.0,6.4,6.0,7.6,7.2,8.8,11.2,9.2,7.6,6.8,5.6,4.8
S7,9.2,8.0,11.6,11.2,10.8,10.4,12.4,11.2,11.2,11.2,7.2,6.0
S8,7.2,7.6,10.0,12.8,18.8,12.8,13.6,12.4,12.0,10.0,8.0,6.8
S9,10.4,9.2,13.6,14.8,16.4,16.0,16.0,11.6,13.2,10.4,8.0,6.4


In [12]:
def rSubset(shops, r):
  
    # return list of all subsets of length r
    # to deal with duplicate subsets use 
    # set(list(combinations(shops, r)))
    return list(combinations(shops, r))

In [13]:
list_of_sets = pd.DataFrame()

if __name__ == "__main__":
    shops = vol_av_daily_per_month.index #all the shop names
    
    for r in range(0, 5): #the max amount of shops per set is 4
        for i in range(0,len(rSubset(shops, r))):
            vol_sum = 0
            for j in range(0,r):
                vol_sum = vol_sum + vol_av_daily_per_month[1][rSubset(shops, r)[i][j]] #value of shop 
        
            if vol_sum <= 20: #the max value of one truck is 20 m3
                temp = pd.DataFrame(
                    {'subset': str(rSubset(shops, r)[i])}, index=[0]
                )
                list_of_sets = pd.concat([list_of_sets, temp]) 

In [14]:
list_of_sets = list_of_sets.iloc[1: , :].reset_index()

In [15]:
list_of_sets

,index,subset
0,0,"('S1',)"
1,0,"('S2',)"
2,0,"('S3',)"
3,0,"('S4',)"
4,0,"('S5',)"
...,...,...
668,0,"('S4', 'S5', 'S18', 'S19')"
669,0,"('S4', 'S5', 'S19', 'S20')"
670,0,"('S5', 'S6', 'S18', 'S19')"
671,0,"('S5', 'S6', 'S19', 'S20')"


# Combining sets

In [16]:
matches = vol_av_daily_per_month.index

In [17]:
db = pd.DataFrame([0]) #creating a dataframe with the right structure
n = 3 #choosing which shop

for i in range(0,len(list_of_sets)):
    db[i] = pd.DataFrame([list_of_sets['subset'][i].find(matches[n])]) #find which sets contain this shop
    
tr = db.transpose().rename(columns={0:'subset'}) #creating a true false list for each set, the find function works differently
tr[tr['subset'] < 0] = False
tr[tr['subset'] > 0] = True
tr

,subset
0,False
1,False
2,False
3,True
4,False
...,...
668,True
669,True
670,False
671,False


In [18]:
contains = pd.DataFrame(list_of_sets[tr]['subset'].dropna()) #list of combinations with this shop
contains['subset']

3                         ('S4',)
22                   ('S1', 'S4')
40                   ('S2', 'S4')
57                   ('S3', 'S4')
74                   ('S4', 'S5')
                  ...            
651    ('S2', 'S4', 'S18', 'S19')
652    ('S2', 'S4', 'S19', 'S20')
664     ('S3', 'S4', 'S5', 'S19')
668    ('S4', 'S5', 'S18', 'S19')
669    ('S4', 'S5', 'S19', 'S20')
Name: subset, Length: 119, dtype: object

In [19]:
list_of_sets_per_shop = pd.DataFrame() #creating a dataframe to store the lists into

for n in range(0,20):

    for i in range(0,len(list_of_sets)):
        db[i] = pd.DataFrame([list_of_sets['subset'][i].find(matches[n])]) #this still doesn't work, it sees 11, 12 etc as 1 and 20 as 2
    
    
    tr = db.transpose().rename(columns={0:'S'+str(n+1)+'_'})
    tr[tr['S'+str(n+1)+'_'] < 0] = False
    tr[tr['S'+str(n+1)+'_'] > 0] = True
    
    #contains = pd.DataFrame(list_of_sets[tr]['subset'].dropna())
    
    list_of_sets_per_shop = pd.concat([list_of_sets_per_shop, tr['S'+str(n+1)+'_']], axis=1, join="outer")
    

In [20]:
list_of_sets_per_shop.head(20) #a list with for each shop the sets it is in

,S1_,S2_,S3_,S4_,S5_,S6_,S7_,S8_,S9_,S10_,S11_,S12_,S13_,S14_,S15_,S16_,S17_,S18_,S19_,S20_
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
9,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


In [21]:
comb = []
for n in range(0,20):
    for i in range(0,20):
        if(list_of_sets_per_shop['S'+str(n+1)][i] == True): #return for each i the shop for which it is true
            comb = comb + [i] 
    

comb.sort()
list_of_sets.iloc[comb]

KeyError: 'S1'

In [ ]:
arra = [1,2,3]
list_of_sets.iloc[arra]

In [ ]:
comb = str() #creating empty string to fill
list_of_comb = pd.DataFrame() #creating empty dataframe to fill

for n in range(0, len(matches)):
    for i in range(0,len(list_of_sets)): #here it goes through all the combinations for each shop
        
        #if the shop in this combination are not yet in the list of combinations add 
        #for this make a matrix of the sets in the list and the shops that are already in the combination
        if list_of_sets_per_shop['S'+str(n+1)][i] not in 
            print(i) #print combination 
            #maybe make a sperate list of the sets and off the shops already in the sets
        
        
        (all(x in list_of_sets['subset'][i] for x in matches[n]) and any(x not in list_of_comb['subset'][i] for x in matches[:n-1])):
            comb = comb + ' ' + list_of_sets['subset'][i]
            temp = pd.DataFrame(
                {'combinations': str(comb)}, index=[[n, i]]
            )
    list_of_comb = pd.concat([list_of_comb, temp])